In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

In [ ]:
movies.head(1)

In [ ]:
credits.head(1)['cast'].values

In [ ]:
movies = movies.merge(credits,on="title")

In [ ]:
movies.head(1)

In [ ]:
movies.info()

In [ ]:
movies = movies[['movie_id','genres','id','keywords','title','overview','cast','crew']]

In [ ]:
movies.head()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace= True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
import ast

def convert(obj):
    L = []
    
    # Ensure obj is a string before evaluating
    if isinstance(obj, str):
        try:
            obj = ast.literal_eval(obj)
        except (ValueError, SyntaxError):
            return []  # Return an empty list if conversion fails

    # If obj is already a list, proceed
    if isinstance(obj, list):
        for i in obj:
            if isinstance(i, dict) and 'name' in i:
                L.append(i['name'])
            elif isinstance(i, str):  # If it's a string, append it directly
                L.append(i)

    return L


In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
movies['cast'][0]

In [ ]:
import ast

def convert3(obj):
    L = []
    counter = 0
    # Ensure obj is a string before evaluating
    if isinstance(obj, str):
        try:
            obj = ast.literal_eval(obj)
        except (ValueError, SyntaxError):
            return []  # Return an empty list if conversion fails

    # If obj is already a list, proceed
    if isinstance(obj, list):
        for i in obj:
            if counter!=3:
                if isinstance(i, dict) and 'name' in i:
                    L.append(i['name'])
                    counter+=1
                elif isinstance(i, str):  # If it's a string, append it directly
                    L.append(i)
            else:
                break
    return L


In [ ]:
movies['cast'] = movies['cast'].apply(convert3)

In [ ]:
movies['cast']

In [ ]:
movies.head()

In [ ]:
import ast

def director(obj):
    L = []
    
    # Ensure obj is a string before evaluating
    if isinstance(obj, str):
        try:
            obj = ast.literal_eval(obj)
        except (ValueError, SyntaxError):
            return []  # Return an empty list if conversion fails

    # If obj is already a list, proceed
    if isinstance(obj, list):
        for i in obj:
            if i['job']=='Director':
                if isinstance(i, dict) and 'name' in i:
                    L.append(i['name'])
                break  
            elif isinstance(i, str):  # If it's a string, append it directly
                    L.append(i)
    return L


In [ ]:
movies['crew'] = movies['crew'].apply(director)

In [ ]:
movies['crew']

In [ ]:
movies.head()

In [ ]:
movies['overview'][1]

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies['overview']

In [ ]:
movies.head()

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x]) #to remove space between words

In [ ]:
movies['genres']

In [ ]:
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies['cast']

In [ ]:
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

In [ ]:
movies['title'] = movies['title'].apply(lambda x: ''.join(x) if isinstance(x, list) else x)


In [ ]:
movies['tags'] = movies['overview']+movies['cast']+movies['crew']+movies['keywords']+movies['genres']

In [ ]:
movies.head()

In [ ]:
new_df=movies[['movie_id','title','tags']]

In [ ]:
new_df

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df.head()

In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df['tags'][0]

In [ ]:
new_df.head()

In [ ]:
import sklearn

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors

In [ ]:
vectors[2]

In [ ]:
np.set_printoptions(threshold=np.inf)  # Prevents NumPy truncation
print(cv.get_feature_names_out())

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
stem('in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. samworthington zoesaldana sigourneyweaver jamescameron cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d action adventure fantasy sciencefiction')

In [ ]:
new_df['tags']=new_df['tags'].apply(stem)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similar = cosine_similarity(vectors)

In [ ]:
similar

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]  

    distances = similar[movie_index]
    
    movies_list = sorted(enumerate(distances), key=lambda x: x[1], reverse=True)[1:6]  

    for i in movies_list:
        print(new_df.iloc[i[0]]['title'])  


In [ ]:
recommend('Avatar')

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [ ]:
new_df['title'].values

In [ ]:
pickle.dump(new_df.to_dict(),open('movies_diction.pkl','wb'))

In [ ]:
pickle.dump(similar,open('similarity.pkl','wb'))